# Resilient RDDs

### Introduction

In the last lessons we saw two of the main components of Spark.  We saw that Spark primarily saves it's data to memory, and that Spark stores a dataset distributed across executors, that query and operate on that data in parallel.  Spark calls this distributed data set a Resilient Distributed Dataset (RDD).

Now, storing data distributed across nodes in memory comes with it's challenges -- mainly that if one of the nodes goes down, we'll want a way to recover the lost data, but of course in Spark that data is not saved to disk.  So in this lesson, we'll see how Spark does recover when data when a node goes down.  This should help to explain the *resilient* component of our Resilient Distributed Datasets. 

### Building Fault Tolerance In Memory

So as we know, the principal feature of Spark is that it provides for in memory storage.  Now in memory storage comes with some challenges -- mainly that even thouh we are not saving any updates to disk, we still want our spark cluster to be *fault tolerant*.  This means that even if one of our nodes goes down, we do not want the data on that node to be lost.

Normally, distributed databases achieve this by copying partitions of the data to multiple nodes.  This way if one node goes down, there is still a backup.

> In the diagram below we can see that the `D` movies are copied to two different nodes.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/copied_data.jpg?raw=1" width="40%">

However, there are downsides to this approach:
1. This copying takes up a significant amount of space, and 
2. It requires copying data over a the cluster's network, and oftentimes there may be narrow bandwidth to do so

> The diagram below shows the copying process from one node to the other.  With a lot of data, and narrow bandwidth, this can be a slow process.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/network_slow.jpg?raw=1" width="60%">

In Spark things are done differently.  Instead of copying the data over, from one node to another, Spark instead keeps track of all of the steps to recreate our dataset in the driver node.  So if the node goes down, it can simply reapply those steps.

We'll learn more about this in the next section.

### Getting Setup (On Google Colab)

* Begin by installing some pip packages and the java development kit.

In [ ]:
!pip install pyspark --quiet
!pip install -U -q PyDrive --quiet 
!apt install openjdk-8-jdk-headless &> /dev/null

* Then set the java environmental variable

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

* Then connect to a SparkSession, setting the spark ui port to `4050`.

In [ ]:
from pyspark import SparkContext, SparkConf

conf = SparkConf().set('spark.ui.port', '4050').setAppName("films").setMaster("local[2]")
sc = SparkContext.getOrCreate(conf=conf)

* Then we need to install ngrok which will allow us to place our local spark ui on the web.

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip &> /dev/null
!unzip ngrok-stable-linux-amd64.zip &> /dev/null
get_ipython().system_raw('./ngrok http 4050 &')

* And finally we get a link our Spark UI

In [ ]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

### Viewing the Transformations

Let's get started by using our spark context to create an RDD.

In [ ]:
movies = ['dark knight', 'dunkirk', 'pulp fiction', 'avatar']

In [ ]:
rdd = sc.parallelize(movies)

This time we'll change our data.  We can do so by using the `map` function to capitalize each word in our list of movies.

In [ ]:
rdd.map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

> So above, `movie` represents each title in the list, and we call `title` on each one to capitalize.  We see the results of the transformation with `collect`.

* Seeing it in the Spark UI

Now we have seen how Spark records these transformations by looking at the Spark UI.  Let's take another look.

In [ ]:
sc

<SparkContext master=local[2] appName=films>

> If we click on the Spark UI, and then the most recent Job, we can see the dag.

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/dag-viz.png?raw=1" width="60%">

Now, underneath, Spark logs the transformation more in a more detailed manner than the UI illustrates.  But hopefully we can get the idea, that Spark tracks the transformations made on a dataset, from when it's read from an external dataset, to the ultimate output.  And if a node goes down, Spark can re-execute the steps on just that portion of the data.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/rdd_one_to_two.jpg?raw=1" width="40%">

The other thing to note is that when we apply a transformation to a dataset, we are actually creating a new RDD.  Again, we can see that in the DAG.  

> <img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/dag-viz.png?raw=1" width="40%">

So we are never updating an RDD.  Our RDDs are read only, and whenever we filter or map through a dataset, we are creating a new RDD in the process.  

### Only Coarse Transformations

Because keeping track of every tiny change that happens to a dataset takes some work, Spark limits the kinds of transformations we can apply.  Namely, when we apply changes, we must apply these changes to the *entire* RDD.  For example, above, we capitalized every record with the `map` function.

In [ ]:
rdd.map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk', 'Pulp Fiction', 'Avatar']

Or, with our RDDs, we can also go through every record, and only select those that begin with the letter `d`.

In [ ]:
rdd.filter(lambda movie: movie[0] == 'd').collect()

['dark knight', 'dunkirk']

> But this is still considered an operation on the entire dataset because we search through every record.

The point from above, though is that whether we use `map` or `filter`, each step applies to *every* record.  These types of transformations are called **coarse grained transformations** - and these are the only kinds of transformations that Spark allows.  If we were to select individual records and then make changes to them, this would be fine-grained transformations.  

If we want to apply a transformation to a small subset of data, we'll need to first use a filter to select our matching records, and then apply the map function.

In [ ]:
rdd.filter(lambda movie: movie[0] == 'd').map(lambda movie: movie.title()).collect()

['Dark Knight', 'Dunkirk']

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/filter_map.jpg?raw=1" width="40%">

### Summary

So we saw in this lesson that Spark achieves fault tolerance by keeping a recording of the transformations needed to recreate our data.  Because the RDDs are read only, so when we transform our data, really we are creating a new RDD.  And Spark keeps track of the steps necessary to go from one transformation to the other.

<img src="https://github.com/jigsawlabs-student/pyspark-rdds/blob/main/rdd_one_to_two.jpg?raw=1" width="40%">

To make recording these steps easier, on Spark RDDs, we can only apply coarse grained transformations, which apply to the entire Spark dataset.  We'll learn more of these transformations in the following lesson, but to start, `map` which applies the same change to every record, and `filter` which selects from a set of elements are two coarse grained transformations.

### Resources

[Spark Debugging Minibook](https://cs.famaf.unc.edu.ar/~damian/tmp/bib/Mini%20eBook%20-%20Apache%20Spark%20Monitoring%20and%20Debugging.pdf)

[Presenting RDDs](https://www.usenix.org/system/files/conference/nsdi12/nsdi12-final138.pdf)

[RDD Programming Guide](https://spark.apache.org/docs/latest/rdd-programming-guide.html)

* [RDDs Simplified](https://vishnuviswanath.com/spark_rdd)

* [Databricks RDDs](https://databricks.com/glossary/what-is-rdd)

[Databricks best practices](https://databricks.gitbooks.io/databricks-spark-knowledge-base/content/index.html)